In [1]:
import numpy as np
import netCDF4 as nc
import xarray as xr
import matplotlib.pyplot as plt
import time
from Helper_fun import time_intersection, generate_temp_range
import os
import datetime as dt
import math
import numba as nb

In [2]:
fast_mode=False
Write_csv=True

In [3]:
if os.environ['WORK_DIR']!='':
    WORK_DIR=os.environ['WORK_DIR']
    PyFLEXTRKR_LIB_DIR= os.environ['PyFLEXTRKR_LIB_DIR']
else:
    raise ValueError("PyFLEXTRKR_LIB_DIR environmental variable is empty or doesn't exist") 

In [4]:
job_output_dir=WORK_DIR+'/Job_output'#'/Old_results_storage/Run12.11_all_t_ranges'

In [6]:
class cloud:
    # def __new__(self, *args, **kwargs):
    #     return super().__new__(self)
    def __init__(self,cloud_id):
        self.id=cloud_id
        self.crit_fraction=0.1
        # Bools inidicating if the cloud has been liquid at any point
        self.is_liq: bool =False
        self.is_mix: bool =False
        self.is_ice: bool =False
        # Max and min cloud size in pixels
        self.max_size_km: float =0.0
        self.max_size_px: int = 0
        self.min_size_km: float =510.0e6
        self.min_size_px: int = 3717*3717
        
        #Variables giving the first and last 4 timesteps (1 hour) of the cloud ice fraction - both arrays run in the same time direction start: [1 , 2 , 3 , 4] ... end: [1 , 2 , 3 , 4]
        self.start_ice_fraction_arr=np.empty(4)
        self.end_ice_fraction_arr=np.empty(4)
        # self.ice_fraction_arr=np.empty(max_timesteps)
        self.ice_fraction_list=[]

        self.max_water_fraction:float=0.0
        self.max_ice_fraction:float=0.0

        self.track_start_time: dt.datetime=None
        self.track_end_time: dt.datetime=None
        self.track_length = None

        self.glaciation_start_time: dt.datetime=None
        self.glaciation_end_time: dt.datetime=None

        self.n_timesteps=None

        self.sum_cloud_lat=0.0
        self.sum_cloud_lon=0.0
        self.avg_cloud_lat=None
        self.avg_cloud_lon=None

        self.sum_cloud_size_km=0.0
        self.avg_cloud_size_km=None

        self.sum_cloud_size_px=0.0
        self.avg_cloud_size_px=None

        self.n_timesteps_no_cloud=0
        self.terminate_cloud=False

        
    def __str__(self):
        return f"{self.is_liq},{self.is_mix},{self.is_ice},"

    def update_status(self,time: dt.datetime, cloud_values: np.array,cloud_lat,cloud_lon,lat_resolution,lon_resolution):
        cloud_size_px=cloud_values.shape[0]
        if cloud_size_px:
            self.n_timesteps_no_cloud=0
            water_fraction=float(np.count_nonzero(cloud_values==1))/float(cloud_size_px)
            # ice_fraction=float(np.count_nonzero(cloud_values==2))/float(cloud_size_px)
            ice_fraction=1-water_fraction
            # assert math.isclose(water_fraction+ice_fraction,1)
            #print(water_fraction)
            #print(water_fraction)
            if not (self.track_start_time):
                self.track_start_time=time
                self.n_timesteps=1
            else:
                self.n_timesteps+=1
            if self.n_timesteps<=4:
                self.start_ice_fraction_arr[self.n_timesteps-1]=ice_fraction
            #Check and set type of cloud
            if water_fraction>1-self.crit_fraction:
                self.is_liq=True
            elif water_fraction>self.crit_fraction:
                self.is_mix=True
            else:
                self.is_ice=True

            cloud_size_km=lat_resolution*lon_resolution*cloud_size_px*np.cos(np.deg2rad(cloud_lat))*111.321*111.111

            self.max_size_km=max(self.max_size_km, cloud_size_km)
            self.min_size_km=min(self.min_size_km, cloud_size_km)

            self.max_size_px=max(self.max_size_px, cloud_size_px)
            self.min_size_px=min(self.min_size_px, cloud_size_px)

            self.sum_cloud_size_px+=cloud_size_px
            self.avg_cloud_size_px=self.sum_cloud_size_px/self.n_timesteps
            
            self.sum_cloud_size_km+=cloud_size_km
            self.avg_cloud_size_km=self.sum_cloud_size_km/self.n_timesteps
            

            # I assume that water_frac+ice_frac=1
            
            self.max_water_fraction=max(self.max_water_fraction, water_fraction)
            self.max_ice_fraction=max(self.max_ice_fraction, 1-water_fraction)

            self.sum_cloud_lat+=cloud_lat
            self.sum_cloud_lon+=cloud_lon
            self.avg_cloud_lat=self.sum_cloud_lat/self.n_timesteps
            self.avg_cloud_lon=self.sum_cloud_lon/self.n_timesteps

            self.track_end_time=time
            self.track_length=self.track_end_time-self.track_start_time
            
            self.end_ice_fraction_arr[0:3]=self.end_ice_fraction_arr[1:4]
            self.end_ice_fraction_arr[3]=ice_fraction
            
            # self.ice_fraction_arr[n_timesteps]=ice_fraction
            self.ice_fraction_list.append(ice_fraction)
            
            
    def update_missing_cloud(self):
        if self.track_end_time and (not self.terminate_cloud):
            self.n_timesteps_no_cloud+=1
            if self.n_timesteps_no_cloud > 1:
                self.terminate_cloud=True

In [7]:
@nb.njit
def extract_cloud_coordinates(cloudtracknumber_field,cloud_id_in_field, max_size):
    # Define the dictionary with the appropriate types
    loc_hash_map_cloud_numbers={j: (0,np.zeros((2,max_size),dtype=np.int16)) for j in cloud_id_in_field}
    # # Traverse the 3D array
    # for i in cloud_id_in_field:
    #     loc_hash_map_cloud_numbers[val] = (0,np.empty((2,max_size),dtype=np.int16))
    for row in range(cloudtracknumber_field.shape[1]):
        for col in range(cloudtracknumber_field.shape[2]):
            val = cloudtracknumber_field[0, row, col]
            if val !=0:
                ind,cord=loc_hash_map_cloud_numbers[val]
                if ind<=max_size:
                    cord[:,ind]=np.asarray([row,col],dtype=np.int16)
                    ind+=1
                    # print(ind)
                    loc_hash_map_cloud_numbers[val] = (ind,cord)
    return loc_hash_map_cloud_numbers
    # return loc_hash_map_cloud_numbers

In [8]:
#2,5,10,15,38
t_deltas = [5]
min_temp_array, max_temp_array = generate_temp_range(t_deltas)

In [9]:
agg_fact_list=[3]#[1,2,5,10]#[2,3,5,10]#1,

In [10]:
cloud_list_agg=[]
for agg_fact in agg_fact_list:
    cloud_list=[]
    sum_append_cloud=dt.timedelta(seconds=0)
    sum_analyze_cloud=dt.timedelta(seconds=0)
    sum_load_and_analyze_cloud=dt.timedelta(seconds=0)
    sum_load_track_variables=dt.timedelta(seconds=0)
    sum_data_loading=dt.timedelta(seconds=0)
    sum_current_cloud_load=dt.timedelta(seconds=0)
    sum_update_status_time=dt.timedelta(seconds=0)
    for temp_ind in range(len(min_temp_array)):
        # loop_start_time=dt.datetime.now()
        loop_start_time=dt.datetime.now()
        min_temp, max_temp = min_temp_array[temp_ind],max_temp_array[temp_ind]
        #Load datasets
        try:
            if agg_fact!=0:
                current_iteration_dir= job_output_dir+f'/T-{abs(round(min_temp))}-{abs(round(max_temp))}-agg-{agg_fact}'
            else:
                current_iteration_dir= job_output_dir+f'/T-{abs(round(min_temp))}-{abs(round(max_temp))}/'
            cloudtrack_data = nc.Dataset(current_iteration_dir+'/pixel_path_tracking/20040201.1415_20040201.2000/cloudtracks_20040201_183000.nc')
            trackstats_data=nc.Dataset(current_iteration_dir+'/stats/trackstats_final_20040201.1415_20040201.2000.nc')
            tracknumbers_data=nc.Dataset(current_iteration_dir+'/stats/tracknumbers_20040201.1415_20040201.2000.nc')
        except: #Exception as inst:
            print(f"Skipping {min_temp} to {max_temp}")
            # print(type(inst))    # the exception type

            # print(inst.args)     # arguments stored in .args

            # print(inst)          # __str__ allows args to be printed directly,

                                # but may be overridden in exception subclasses
            cloud_list.append([])
            continue
        #Load relevant data from datasets into local variables
        cloudtracknumber_field=cloudtrack_data.variables['tracknumber'][:,:,:]
        cph_field=cloudtrack_data.variables['cph'][:,:,:]
        n_tracks=trackstats_data.variables['track_duration'].shape[0]
        basetimes=tracknumbers_data.variables['basetimes'][:]
        lat=cloudtrack_data.variables['lat'][:]
        lon=cloudtrack_data.variables['lon'][:]
        lat_resolution=(lat.max()-lat.min())/len(lat)
        lon_resolution=(lon.max()-lon.min())/len(lon)
        trackstats_data.close()
        tracknumbers_data.close()
        cloudtrack_data.close()
        #FIX CLOUD TIMES
        # print(n_tracks)
        # append_start_time=dt.datetime.now()
        append_start_time=dt.datetime.now()
        sum_load_track_variables+=append_start_time-loop_start_time
        # print(append_start_time-loop_start_time)
        cloud_list.append([cloud(f'{temp_ind}_{i}') for i in range(n_tracks)])
        append_end_time=dt.datetime.now()
        sum_append_cloud+=append_end_time-append_start_time
        # print(append_end_time-append_start_time)
        print(f'Analyzing T: {min_temp} to {max_temp} Agg={agg_fact}')
        for unix_time in basetimes:
            data_loading_start_time=dt.datetime.now()
            time=dt.datetime.utcfromtimestamp(unix_time)
            time_str=time.strftime("%Y%m%d_%H%M%S")
            print(f'{min_temp} to {max_temp} Loading {time_str}')
            cloudtrack_fp = current_iteration_dir+f'/pixel_path_tracking/20040201.1415_20040201.2000/cloudtracks_{time_str}.nc'
            cloudtrack_data = nc.Dataset(cloudtrack_fp) 
            cloudtracknumber_field=cloudtrack_data.variables['tracknumber'][:,:,:].data
            cph_field=cloudtrack_data.variables['cph'][:,:,:]
            cloudtrack_data.close()
            analysis_start_time=dt.datetime.now()
            sum_data_loading+=analysis_start_time-data_loading_start_time
            cloud_id_in_field , counts = np.unique(cloudtracknumber_field, return_counts=True)
            counts=counts[cloud_id_in_field!=0]
            cloud_id_in_field=cloud_id_in_field[cloud_id_in_field!=0]
            max_allowed_cloud_size_px= 20000 if fast_mode else counts.max()
            hash_map_cloud_numbers=extract_cloud_coordinates(cloudtracknumber_field,cloud_id_in_field,max_allowed_cloud_size_px)#counts.max())
            if max_allowed_cloud_size_px>1000000:
                print(np.where(counts, counts==counts.max()))
            for track_number in cloud_id_in_field:
                current_cloud_select_time=dt.datetime.now()
                try:
                    current_cloud=cloud_list[temp_ind][track_number-1]
                except:
                    print(f"Error: {temp_ind,track_number,len(cloud_list[temp_ind])}")
                    continue
                start_update_status_time=dt.datetime.now()
                if (not current_cloud.terminate_cloud):
                    sum_current_cloud_load+=start_update_status_time-current_cloud_select_time
                    #TODO:SPEED UP NEXT TWO LINES (set_cloud_values and update_status)
                    ind ,cord=hash_map_cloud_numbers[track_number]
                    cloud_location_ind=[cord[0,:ind],cord[1,:ind]]
                    if cloud_location_ind[0].size!=0:
                        avg_lat_ind=int(round(np.mean(cloud_location_ind[0])))
                        avg_lon_ind=int(round(np.mean(cloud_location_ind[1])))
                        #TODO:SPEED UP NEXT TWO LINES (set_cloud_values and update_status)
                        cloud_values=cph_field[0,cloud_location_ind[0].T,cloud_location_ind[1].T]
                        current_cloud.update_status(time,cloud_values,lat[avg_lat_ind],lon[avg_lon_ind],lat_resolution,lon_resolution)
                    else:
                        current_cloud.update_missing_cloud()
                sum_update_status_time+=dt.datetime.now()-start_update_status_time
            sum_analyze_cloud+=dt.datetime.now()-analysis_start_time
            
            # cloudtrack_data.close()
        sum_load_and_analyze_cloud+=dt.datetime.now()-append_end_time
        # print(dt.datetime.now()-append_end_time)

    cloud_list_agg.append(cloud_list)


Analyzing T: -5 to 0 Agg=3
-5 to 0 Loading 20040201_141500
-5 to 0 Loading 20040201_143000
-5 to 0 Loading 20040201_144500
-5 to 0 Loading 20040201_150000
-5 to 0 Loading 20040201_151500
-5 to 0 Loading 20040201_153000
-5 to 0 Loading 20040201_154500
-5 to 0 Loading 20040201_160000
-5 to 0 Loading 20040201_161500
-5 to 0 Loading 20040201_163000
-5 to 0 Loading 20040201_164500
-5 to 0 Loading 20040201_170000
-5 to 0 Loading 20040201_171500
-5 to 0 Loading 20040201_173000
-5 to 0 Loading 20040201_174500
-5 to 0 Loading 20040201_180000
-5 to 0 Loading 20040201_181500
-5 to 0 Loading 20040201_183000
-5 to 0 Loading 20040201_184500
-5 to 0 Loading 20040201_190000
-5 to 0 Loading 20040201_191500
-5 to 0 Loading 20040201_193000
-5 to 0 Loading 20040201_194500
-5 to 0 Loading 20040201_200000
Analyzing T: -10 to -5 Agg=3
-10 to -5 Loading 20040201_141500
-10 to -5 Loading 20040201_143000
-10 to -5 Loading 20040201_144500
-10 to -5 Loading 20040201_150000
-10 to -5 Loading 20040201_151500
-10 to

In [11]:
print(sum_append_cloud)
print(sum_analyze_cloud)
print(sum_load_and_analyze_cloud)
print(sum_load_track_variables)
print(sum_data_loading)
print(sum_current_cloud_load)
print(sum_update_status_time)

0:00:00.011519
0:01:14.329384
0:01:32.467609
0:00:01.410100
0:00:18.137019
0:00:00.023006
0:00:01.698441


In [13]:
import pandas as pd
import datetime as dt
lat_resolution=(lat.max()-lat.min())/len(lat)
lon_resolution=(lon.max()-lon.min())/len(lon)
columns=["is_liq","is_mix","is_ice","max_water_frac","max_ice_fraction","avg_size[km]","max_size[km]","min_size[km]","avg_size[px]","max_size[px]","min_size[px]","track_start_time","track_length","glaciation_start_time","glaciation_end_time","avg_lat","avg_lon","start_ice_fraction","end_ice_fraction","ice_frac_hist"]
datapoints_per_cloud=len(columns)
for agg_ind in range(len(agg_fact_list)):
    cloud_list = cloud_list_agg[agg_ind]
    agg_fact=agg_fact_list[agg_ind]
    for dt_ind in range(len(t_deltas)):
        # TODO Change iteration method in for loop so that for each dt it only goes through the relevant parts of cloud list instead of the whole
        for cloud_list_ind in range(len(cloud_list)):
            temp_delta=t_deltas[dt_ind]
            cloudinfo_df=pd.DataFrame(index=range(len(cloud_list[cloud_list_ind])),columns=columns)
            for cloud_ind in range(len(cloud_list[cloud_list_ind])):
                current_cloud=cloud_list[cloud_list_ind][cloud_ind]
                # current_cloud.max_size_km=current_cloud.max_size_px * lat_resolution*lon_resolution*np.cos(np.deg2rad(current_cloud.avg_cloud_lat))*111.321*111.111
                # current_cloud.min_size_km=current_cloud.min_size_px * lat_resolution*lon_resolution*np.cos(np.deg2rad(current_cloud.avg_cloud_lat))*111.321*111.111

                cloudinfo_df.iloc[cloud_ind]=[current_cloud.is_liq,current_cloud.is_mix,current_cloud.is_ice,current_cloud.max_water_fraction,current_cloud.max_ice_fraction,current_cloud.avg_cloud_size_km,current_cloud.max_size_km,current_cloud.min_size_km,current_cloud.avg_cloud_size_px ,current_cloud.max_size_px,current_cloud.min_size_px,current_cloud.track_start_time,current_cloud.track_length,current_cloud.glaciation_start_time , current_cloud.glaciation_end_time, current_cloud.avg_cloud_lat, current_cloud.avg_cloud_lon, current_cloud.start_ice_fraction_arr, current_cloud.end_ice_fraction_arr, current_cloud.ice_fraction_list]
            min_temp, max_temp = min_temp_array[cloud_list_ind],max_temp_array[cloud_list_ind]

            output_dir=WORK_DIR+f"/Cloud_analysis/T_{abs(round(min_temp))}_{abs(round(max_temp))}_agg_{agg_fact}_tracknumber"
            print("Writing to ",output_dir)
            output_dir_parq=output_dir+".parquet"
            cloudinfo_df.to_parquet(output_dir_parq)
            if Write_csv:
                output_dir_csv=output_dir+".csv"
                cloudinfo_df.to_csv(output_dir_csv)

Writing to  /cluster/work/climate/dnikolo//Cloud_analysis/T_5_0_agg_3_tracknumber
Writing to  /cluster/work/climate/dnikolo//Cloud_analysis/T_10_5_agg_3_tracknumber
Writing to  /cluster/work/climate/dnikolo//Cloud_analysis/T_15_10_agg_3_tracknumber
Writing to  /cluster/work/climate/dnikolo//Cloud_analysis/T_20_15_agg_3_tracknumber
Writing to  /cluster/work/climate/dnikolo//Cloud_analysis/T_25_20_agg_3_tracknumber
Writing to  /cluster/work/climate/dnikolo//Cloud_analysis/T_30_25_agg_3_tracknumber
Writing to  /cluster/work/climate/dnikolo//Cloud_analysis/T_35_30_agg_3_tracknumber


Used for testing

In [ ]:
# import pandas as pd
# import datetime as dt
# lat_resolution=(lat.max()-lat.min())/len(lat)
# lon_resolution=(lon.max()-lon.min())/len(lon)
# columns=["is_liq","is_mix","is_ice","max_water_frac","max_ice_fraction","avg_size[km]","max_size[km]","min_size[km]","max_size[px]","min_size[px]","track_start_time","track_length","glaciation_start_time","glaciation_end_time","avg_lat","avg_lon", "start_ice_fraction","end_ice_fraction"]
# datapoints_per_cloud=len(columns)
# for agg_ind in range(len(agg_fact_list)):
#     cloud_list = cloud_list_agg[agg_ind]
#     agg_fact=agg_fact_list[agg_ind]
#     for dt_ind in range(len(t_deltas)):
#         # TODO Change iteration method in for loop so that for each dt it only goes through the relevant parts of cloud list instead of the whole
#         for cloud_list_ind in range(len(cloud_list)):
#             temp_delta=t_deltas[dt_ind]
#             cloudinfo_df=pd.DataFrame(index=range(len(cloud_list[cloud_list_ind])),columns=columns)
#             for cloud_ind in range(len(cloud_list[cloud_list_ind])):
#                 current_cloud=cloud_list[cloud_list_ind][cloud_ind]
#                 # current_cloud.max_size_km=current_cloud.max_size_px * lat_resolution*lon_resolution*np.cos(np.deg2rad(current_cloud.avg_cloud_lat))*111.321*111.111
#                 # current_cloud.min_size_km=current_cloud.min_size_px * lat_resolution*lon_resolution*np.cos(np.deg2rad(current_cloud.avg_cloud_lat))*111.321*111.111

#                 cloudinfo_df.iloc[cloud_ind]=[current_cloud.is_liq,current_cloud.is_mix,current_cloud.is_ice,current_cloud.max_water_fraction,current_cloud.max_ice_fraction,current_cloud.avg_cloud_size_km,current_cloud.max_size_km,current_cloud.min_size_km,current_cloud.max_size_px,current_cloud.min_size_px,current_cloud.track_start_time,current_cloud.track_length,current_cloud.glaciation_start_time , current_cloud.glaciation_end_time, current_cloud.avg_cloud_lat, current_cloud.avg_cloud_lon, current_cloud.start_ice_fraction_arr, current_cloud.end_ice_fraction_arr]
#             min_temp, max_temp = min_temp_array[cloud_list_ind],max_temp_array[cloud_list_ind]

#             output_dir=WORK_DIR+f"/Cloud_analysis/T_{abs(round(min_temp))}_{abs(round(max_temp))}_agg_{agg_fact}_tracknumber"
#             output_dir_csv=output_dir+".csv"
#             output_dir_parq=output_dir+".parquet"
#             print("Writing to ",output_dir_csv)
#             # cloudinfo_df.to_csv(output_dir_csv)
#             cloudinfo_xr=cloudinfo_df.to_parquet(output_dir_parq)
#             #  cloudinfo_xr.to_netcdf(output_dir_nc)

In [ ]:
# cloudinfo_xr

In [ ]:

# n_track_arr=np.empty(len(min_temp_array))
# n_glac_arr=np.empty(len(min_temp_array))

# for i in range(len(cloud_list)):
#     n_track_arr[i]=len(cloud_list[i])

#     glaciation_counter=0
#     only_ice_counter=0
#     only_mix_counter=0
#     only_liq_counter=0
#     cirrus_counter=0
#     for cloud in cloud_list[i]:
#         print(f"Cloud max ice fraction: {round(cloud.max_ice_fraction,2)}; Cloud max liq fraction: {round(cloud.max_water_fraction,2)} " )
#         if cloud.is_liq+cloud.is_mix+cloud.is_ice==3:
#             glaciation_counter+=1
#         if (not (cloud.is_liq or cloud.is_mix) ) and cloud.is_ice:
#             cirrus_counter+=1
#         if (not (cloud.is_ice or cloud.is_mix) ) and cloud.is_liq:
#             only_liq_counter+=1
#         if (not (cloud.is_liq or cloud.is_ice) ) and cloud.is_mix:
#             only_mix_counter+=1
#         if cloud.is_ice:
#             only_ice_counter+=1
#     n_glac_arr[i]=only_mix_counter# glaciations
        
# print(f"N glaciations: {glaciation_counter}; N only cirrus: {cirrus_counter}; N only liquid: {only_liq_counter}; N only mix: {only_mix_counter}; N ice at some point: {only_ice_counter}")

In [ ]:
# from datetime import datetime, timedelta

# # Function to generate time strings
# def iterate_time(start_date, end_date):
#     current_time = start_date
#     while current_time <= end_date:
#         # Format the time as YYYYMODD_HHMMSS
#         time_str = current_time.strftime("%Y%m%d_%H%M%S")
#         print(time_str)
#         # Increment the time by 15 minutes
#         current_time += timedelta(minutes=15)

# # Example usage
# start_date = datetime(2024, 10, 15, 9, 0, 0)  # Example start date: 15th Oct 2024, 9:00:00
# end_date = datetime(2024, 10, 15, 12, 0, 0)    # Example end date: 15th Oct 2024, 12:00:00

# iterate_time(start_date, end_date)




In [ ]:
# import matplotlib.pyplot as plt
# # Create labels for the x-axis (temperature ranges)


# labels = [f"{min_temp_array[i]} to {max_temp_array[i]}" for i in range(len(min_temp_array))]

# # Create the bar graph
# plt.figure(figsize=(10,6))
# plt.bar(labels, n_track_arr[np.where(max_temp_array-min_temp_array==5)], color='skyblue',label="N tracks")
# plt.bar(labels, n_glac_arr[np.where(max_temp_array-min_temp_array==5)], color='lightcoral',label="N glaciations")

# # Add title and labels
# plt.title('Number Distribution Across Temperature Ranges', fontsize=14, fontweight='bold')
# plt.xlabel('Temperature Range (°C)', fontsize=12)
# plt.ylabel('Cloud track number', fontsize=12)

# # Rotate x-axis labels for better readability
# plt.xticks(rotation=45)

# # Add gridlines for better visualization
# plt.grid(axis='y', linestyle='--', alpha=0.7)

# # Show the plot
# plt.tight_layout()
# # plt.savefig(f'/net/n2o/wolke_scratch/dnikolo/Glaciation_time_estimatior/Result_graphs/TEST_track_distribution_dt5.png',dpi=400)
# plt.show()

In [ ]:
# cloud_list_agg=[]
# for agg_fact in agg_fact_list:
#     cloud_list=[]
#     sum_append_cloud=dt.timedelta(seconds=0)
#     sum_analyze_cloud=dt.timedelta(seconds=0)
#     sum_load_and_analyze_cloud=dt.timedelta(seconds=0)
#     sum_load_track_variables=dt.timedelta(seconds=0)
#     sum_data_loading=dt.timedelta(seconds=0)
#     sum_current_cloud_load=dt.timedelta(seconds=0)
#     sum_update_status_time=dt.timedelta(seconds=0)
#     for temp_ind in range(len(min_temp_array)):
#         # loop_start_time=dt.datetime.now()
#         loop_start_time=dt.datetime.now()
#         min_temp, max_temp = min_temp_array[temp_ind],max_temp_array[temp_ind]
#         #Load datasets
#         try:
#             if agg_fact!=0:
#                 current_iteration_dir= job_output_dir+f'/T-{abs(round(min_temp))}-{abs(round(max_temp))}-agg-{agg_fact}/'
#             else:
#                 current_iteration_dir= job_output_dir+f'/T-{abs(round(min_temp))}-{abs(round(max_temp))}/'
#             cloudtrack_data = nc.Dataset(current_iteration_dir+'/pixel_path_tracking/20040201.1415_20040201.2000/cloudtracks_20040201_183000.nc')
#             trackstats_data=nc.Dataset(current_iteration_dir+'/stats/trackstats_final_20040201.1415_20040201.2000.nc')
#             tracknumbers_data=nc.Dataset(current_iteration_dir+'/stats/tracknumbers_20040201.1415_20040201.2000.nc')
#         except: #Exception as inst:
#             print(f"Skipping {min_temp} to {max_temp}")
#             # print(type(inst))    # the exception type

#             # print(inst.args)     # arguments stored in .args

#             # print(inst)          # __str__ allows args to be printed directly,

#                                 # but may be overridden in exception subclasses
#             cloud_list.append([])
#             continue
#         #Load relevant data from datasets into local variables
#         cloudtracknumber_field=cloudtrack_data.variables['tracknumber'][:,:,:]
#         cph_field=cloudtrack_data.variables['cph'][:,:,:]
#         n_tracks=trackstats_data.variables['track_duration'].shape[0]
#         basetimes=tracknumbers_data.variables['basetimes'][:]
#         lat=cloudtrack_data.variables['lat'][:]
#         lon=cloudtrack_data.variables['lon'][:]
#         lat_resolution=(lat.max()-lat.min())/len(lat)
#         lon_resolution=(lon.max()-lon.min())/len(lon)
#         trackstats_data.close()
#         tracknumbers_data.close()
#         cloudtrack_data.close()
#         #FIX CLOUD TIMES
#         print(n_tracks)
#         # append_start_time=dt.datetime.now()
#         append_start_time=dt.datetime.now()
#         sum_load_track_variables+=append_start_time-loop_start_time
#         print(append_start_time-loop_start_time)
#         cloud_list.append([cloud(f'{temp_ind}_{i}') for i in range(n_tracks)])
#         append_end_time=dt.datetime.now()
#         sum_append_cloud+=append_end_time-append_start_time
#         print(append_end_time-append_start_time)
#         for unix_time in basetimes:
#             data_loading_start_time=dt.datetime.now()
#             time=dt.datetime.utcfromtimestamp(unix_time)
#             time_str=strftime("%Y%m%d_%H%M%S")
#             print(f'{min_temp} to {max_temp} Loading {time_str}')
#             cloudtrack_fp = current_iteration_dir+f'/pixel_path_tracking/20040201.1415_20040201.2000/cloudtracks_{time_str}.nc'
#             cloudtrack_data = nc.Dataset(cloudtrack_fp) 
#             cloudtracknumber_field=cloudtrack_data.variables['tracknumber'][:,:,:]
#             cph_field=cloudtrack_data.variables['cph'][:,:,:]
#             cloudtrack_data.close()
#             analysis_start_time=dt.datetime.now()
#             sum_data_loading+=analysis_start_time-data_loading_start_time
#             for track_number in range(n_tracks):
#                 current_cloud_select_time=dt.datetime.now()
#                 try:
#                     current_cloud=cloud_list[temp_ind][track_number]
#                 except:
#                     print(f"Error: {temp_ind,track_number,len(cloud_list)}")
#                     exit
#                 start_update_status_time=dt.datetime.now()
#                 if (not current_cloud.terminate_cloud):
#                     sum_current_cloud_load+=start_update_status_time-current_cloud_select_time
#                     #TODO:SPEED UP NEXT TWO LINES (set_cloud_values and update_status)
#                     cloud_location_ind=np.where(cloudtracknumber_field==track_number+1)
#                     if cloud_location_ind[0].size!=0:
#                         avg_lat_ind=int(round(np.mean(cloud_location_ind[1])))
#                         avg_lon_ind=int(round(np.mean(cloud_location_ind[2])))
#                         #TODO:SPEED UP NEXT TWO LINES (set_cloud_values and update_status)
#                         cloud_values=cph_field[cloud_location_ind]
#                         current_cloud.update_status(time,cloud_values,lat[avg_lat_ind],lon[avg_lon_ind],lat_resolution,lon_resolution)
#                     else:
#                         current_cloud.update_missing_cloud()
#                 sum_update_status_time+=dt.datetime.now()-start_update_status_time
#             sum_analyze_cloud+=dt.datetime.now()-analysis_start_time
            
#             # cloudtrack_data.close()
#         sum_load_and_analyze_cloud+=dt.datetime.now()-append_end_time
#         print(dt.datetime.now()-append_end_time)

#     cloud_list_agg.append(cloud_list)
